In [1]:
!pip3 install transformers
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 7.6 MB 47.7 MB/s 
     |████████████████████████████████| 182 kB 65.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 5.3 MB/s 
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
to

In [2]:
data_path = ''
output_path = ''

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import torch
from torchtext.legacy.data import Field, TabularDataset, BucketIterator, Iterator

from transformers import RobertaTokenizer, RobertaModel, AdamW, get_linear_schedule_with_warmup

import warnings
warnings.filterwarnings('ignore')

import logging
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)


In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
model_path = '/content/drive/MyDrive/Work/nlp/Cyber_bullying/model.pkl'
df_train = pd.read_csv('/content/drive/MyDrive/Work/nlp/Cyber_bullying/D1dataset.csv')
df_train.head()

,Word,Class
0,!!! RT @mayasolovely: As a woman you shouldn't...,Non-offensive
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive


In [5]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [6]:
def load_checkpoint(path, model):    
    state_dict = torch.load(path, map_location=device)
    model.load_state_dict(state_dict['model_state_dict'])
    
    return state_dict['valid_loss']

In [7]:
# Model with extra layers on top of RoBERTa
class ROBERTAClassifier(torch.nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(ROBERTAClassifier, self).__init__()
        
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        # self.d1 = torch.nn.Dropout(dropout_rate)
        self.l1 = torch.nn.Linear(768, 64)
        self.bn1 = torch.nn.LayerNorm(64)
        self.d2 = torch.nn.Dropout(dropout_rate)
        self.l2 = torch.nn.Linear(64, 2)
        
    def forward(self, input_ids, attention_mask):
        _, x = self.roberta(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        # x = self.d1(x)
        x = self.l1(x)
        x = self.bn1(x)
        x = torch.nn.Tanh()(x)
        x = self.d2(x)
        x = self.l2(x)
        return x  

In [8]:
model = ROBERTAClassifier()
model = model.to(device)
load_checkpoint(model_path, model)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.1434689634149113

In [22]:
df_train[1:2]

,Word,Class
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive


In [42]:
df_train.shape

(16813, 2)

In [41]:
df_train.head(10)

,Word,Class
0,!!! RT @mayasolovely: As a woman you shouldn't...,Non-offensive
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive
5,"!!!!!!!!!!!!!!!!!!""@T_Madison_x: The shit just...",Offensive
6,"!!!!!!""@__BrighterDays: I can not just sit up ...",Offensive
7,!!!!&#8220;@selfiequeenbri: cause I'm tired of...,Offensive
8,""" &amp; you might not get ya bitch back &amp; ...",Offensive
9,""" @rhythmixx_ :hobbies include: fighting Maria...",Offensive


In [43]:
df_train['Word'][1007]

'&#128514;&#128514;&#8220;@BrandonBeSlydin: My fucking god bruh, the birds from southridge are in front of my house.&#8221;'

In [44]:
df_train['Class'][1007]

'Offensive'

In [24]:
dic = {0:'Non-offensive',1:'Offensive'}

tensor([[    0, 44371, 10541,   787,   119,   459,  2753,  1360,    35,  2143,
           385,  2923,  2569,   734,  2553,  2538,   385, 11538,  1099,    13,
         36201,   179, 13516,  1368,  3540,    11,     5,   112,   620,   317,
         12846,     2,     1,     1,     1,     1,     1,     1,     1,     1]])
--> tensor([1])
Offensive
===
Offensive


In [ ]:
text1 offensive
text2 non offensive


model ---> input(text1) --> output,     offensive

In [37]:
def get_inference():
  inp = input('Enter the text here: ')
  df_test = pd.DataFrame({'Word':[inp],'Class':['Offensive']})

    # df_test = df_train[1:2]
  df_test['Class'] = df_test['Class'].map({'Non-offensive' : 0, 'Offensive' : 1})
  df_test.to_csv('final.csv', index=False)

  # Set tokenizer hyperparameters.
  MAX_SEQ_LEN = 40
  BATCH_SIZE = 16
  PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
  UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

  # Define columns to read.
  label_field = Field(sequential=False, use_vocab=False, batch_first=True)
  text_field = Field(use_vocab=False, 
                    tokenize=tokenizer.encode, 
                    include_lengths=False, 
                    batch_first=True,
                    fix_length=MAX_SEQ_LEN, 
                    pad_token=PAD_INDEX, 
                    unk_token=UNK_INDEX)

  fields = {'Word' : ('Word', text_field), 'Class' : ('Class', label_field)}

  test_data = TabularDataset(path=f"final.csv",  format='CSV',  fields=fields, 
                                                    skip_header=False)
  # Test iterator, no shuffling or sorting required.
  test_iter = Iterator(test_data, batch_size=1, device=device, train=False, 
                      shuffle=False, sort=False)
  
  model.eval()
  with torch.no_grad():
      for (source, target), _ in test_iter:
          print(source)
          print('-->', target)
          mask = (source != PAD_INDEX).type(torch.uint8)
          output = model(source, attention_mask=mask)
          print(dic[torch.argmax(output, axis=-1).tolist()[0]])
          print('===')
          # print(dic[target.tolist()[0]])

In [46]:
get_inference()

Enter the text here: He is doing good and he went to college everyday
tensor([[   0,  894,   16,  608,  205,    8,   37,  439,    7, 1564, 7476,    2,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]])
--> tensor([1])
Non-offensive
===
